In [1]:
#Testing out creating a gif from nii data in python3

In [1]:
import nibabel as nib
from PIL import Image as pImage
import os, sys
import imageio

In [2]:
import numpy as np

In [3]:
def _grayscale_conv(input_array):
    gs_array = 255*(input_array/input_array.max())
    gs_array = np.rint(gs_array)
    return gs_array

In [4]:
def _format_picture(input_array):
    #The input will be an array as read by nib
    #The output will be a PIL Image object
    #gs_array = 255*(input_array/input_array.max())
    #gs_array = np.rint(gs_array)
    pil_image = pImage.fromarray(input_array)
    #Rotate the image (if needed) so the longest side is the width
    if pil_image.size[0] == max(pil_image.size):
        output_pimage = pil_image
    else:
        output_pimage = pil_image.rotate(90,expand=1)
    #Convert the image mode to "LA" for writing
    output_pimage = output_pimage.convert(mode="LA")
    return output_pimage

In [5]:
test_data = '/mnt/keoki/experiments2/Graner/Data/Graner_QA_tools/test_data/fmri/fmri_test_input.nii.gz'

In [6]:
#Load image with nibabel; get image data
input_image = nib.load(test_data)
input_data = input_image.get_data()

In [7]:
#Create slice-wise sections of data
#Pick out the center slice of each time point for each dimension
#(We don't need to know which dimension is which since we're doing all of them)
input_dims = input_data.shape
center_x = round(input_dims[0] / 2.0)
center_y = round(input_dims[1] / 2.0)
center_z = round(input_dims[2] / 2.0)
time_points = input_dims[3]

In [8]:
#Keep only the center slice of each dimension
center_x_image = input_data[center_x, :, :, :]
center_y_image = input_data[:, center_y, :, :]
center_z_image = input_data[:, :, center_z, :]

In [11]:
center_z_image.shape

(128, 128, 50)

In [9]:
#Rescale each group of center slices to 0-255
#This process also squeezes the arrays down to 3 dimensions
center_x_image = _grayscale_conv(center_x_image)
center_y_image = _grayscale_conv(center_y_image)
center_z_image = _grayscale_conv(center_z_image)

In [22]:
one_slice=center_x_image[:,:,1]

In [44]:
max(one_slice.shape)

128

In [26]:
tracker_rows = np.zeros((128, 5))
tracker_rows[:]=255

In [47]:
tracker_rows.shape[1]

5

In [34]:
result = np.concatenate((one_slice, tracker_rows), axis=1)
result.shape

(128, 73)

In [35]:
pil_image = pImage.fromarray(one_slice)

In [40]:
pil_image.data[0,1]

AttributeError: 'Image' object has no attribute 'data'

In [120]:
#For each of the three dimensions, for each timepoint in that dimension,
#extract the slice and create a formatted PIL Image version of it
dim_count = 0
for subset in [center_x_image, center_y_image, center_z_image]:
    slice_files = []
    #Create a temporary png file of the center slice at each timepoint
    for slice_num in range(subset.shape[2]):
        this_slice = subset[:,:,slice_num]
        slice_to_write = _format_picture(this_slice)
        slice_to_write.save(os.path.join(os.getcwd(), 'temp_slice_{}_{}.png'.format(dim_count,slice_num)))
        slice_files.append(os.path.join(os.getcwd(), 'temp_slice_{}_{}.png'.format(dim_count,slice_num)))
    #Create a gif of the center slice pictures
    images = []
    for filename in slice_files:
        images.append(imageio.imread(filename))
    output_gif = os.path.join(os.getcwd(), 'center_slice_gif_{}.gif'.format(dim_count))
    imageio.mimsave(output_gif, images, duration=0.2)
    dim_count = dim_count + 1

In [48]:
thing = np.arange(50)

In [53]:
np.ceil((thing)*(128.0/50.0))

array([   0.,    3.,    6.,    8.,   11.,   13.,   16.,   18.,   21.,
         24.,   26.,   29.,   31.,   34.,   36.,   39.,   41.,   44.,
         47.,   49.,   52.,   54.,   57.,   59.,   62.,   64.,   67.,
         70.,   72.,   75.,   77.,   80.,   82.,   85.,   88.,   90.,
         93.,   95.,   98.,  100.,  103.,  105.,  108.,  111.,  113.,
        116.,  118.,  121.,  123.,  126.])

In [54]:
rows_to_add = np.zeros((128,5))

In [58]:
rows_to_add[0:6,:] = 255

In [59]:
rows_to_add[0,:]

array([ 255.,  255.,  255.,  255.,  255.])

In [60]:
np.rot90(rows_to_add).shape

(5, 128)